# Importing libraries

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Changing the way pandas displays floats

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Importing dataset

In [3]:
suicides = pd.read_csv('/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/master.csv')

In [4]:
suicides.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71000,Albania1987,nan,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19000,Albania1987,nan,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83000,Albania1987,nan,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59000,Albania1987,nan,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28000,Albania1987,nan,"2,156,624,900",796,Boomers


# Checking dtypes

In [5]:
suicides.dtypes

country                object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
HDI for year          float64
 gdp_for_year ($)      object
gdp_per_capita ($)      int64
generation             object
dtype: object

# Renaming columns 

In [6]:
suicides.rename(columns={" gdp_for_year ($) ": "GDP/Year", "gdp_per_capita ($)": "GDP/Capita", "country": "Country",
                        "year": "Year", "sex": "Gender", "suicides_no": "Num_Suicides", "age": "Age", "population":
                        "Population", "suicides/100k pop": "Suicides/100kPop", "generation": "Generation", "HDI for year":
                         "HDI/Year"}, inplace=True)

In [7]:
suicides.head()

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,country-year,HDI/Year,GDP/Year,GDP/Capita,Generation
0,Albania,1987,male,15-24 years,21,312900,6.71000,Albania1987,nan,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19000,Albania1987,nan,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83000,Albania1987,nan,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59000,Albania1987,nan,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28000,Albania1987,nan,"2,156,624,900",796,Boomers


In [8]:
# Dropping this column, redundant information
suicides.drop("country-year", axis=1, inplace=True)

In [9]:
# Re-formatting the columns GDP/Year so it can be converted to float
suicides["GDP/Year"] = suicides["GDP/Year"].apply(lambda x: x.replace(",", ""))

In [10]:
suicides = suicides.astype({"GDP/Year": 'float'})

In [11]:
suicides.dtypes

Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
HDI/Year            float64
GDP/Year            float64
GDP/Capita            int64
Generation           object
dtype: object

In [12]:
suicides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 11 columns):
Country             27820 non-null object
Year                27820 non-null int64
Gender              27820 non-null object
Age                 27820 non-null object
Num_Suicides        27820 non-null int64
Population          27820 non-null int64
Suicides/100kPop    27820 non-null float64
HDI/Year            8364 non-null float64
GDP/Year            27820 non-null float64
GDP/Capita          27820 non-null int64
Generation          27820 non-null object
dtypes: float64(3), int64(4), object(4)
memory usage: 2.3+ MB


In [13]:
suicides.shape[0] - suicides['HDI/Year'].isnull().sum()

8364

In [14]:
# Since we have so many null values and we won't be using this column, we decided to drop it
suicides.drop("HDI/Year", axis=1, inplace=True)

In [15]:
# Checking null values 
suicides.isnull().sum()

Country             0
Year                0
Gender              0
Age                 0
Num_Suicides        0
Population          0
Suicides/100kPop    0
GDP/Year            0
GDP/Capita          0
Generation          0
dtype: int64

In [16]:
suicides.describe()

,Year,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita
count,27820.00000,27820.00000,27820.00000,27820.00000,27820.00000,27820.00000
mean,2001.25838,242.57441,1844793.61740,12.81610,445580969025.72711,16866.46441
std,8.46906,902.04792,3911779.44176,18.96151,1453609985940.91626,18887.57647
min,1985.00000,0.00000,278.00000,0.00000,46919625.00000,251.00000
25%,1995.00000,3.00000,97498.50000,0.92000,8985352832.00000,3447.00000
50%,2002.00000,25.00000,430150.00000,5.99000,48114688201.00000,9372.00000
75%,2008.00000,131.00000,1486143.25000,16.62000,260202429150.00000,24874.00000
max,2016.00000,22338.00000,43805214.00000,224.97000,18120714000000.00000,126352.00000


In [17]:
suicides[suicides['Num_Suicides'] == 0][0:3]

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
9,Albania,1987,female,5-14 years,0,311000,0.00000,2156624900.00000,796,Generation X
10,Albania,1987,female,55-74 years,0,144600,0.00000,2156624900.00000,796,G.I. Generation
11,Albania,1987,male,5-14 years,0,338200,0.00000,2156624900.00000,796,Generation X


In [18]:
driver = 'mysql+pymysql'
host = "34.90.32.189"
username = 'root'
password = '123456789'
db = "suicides1"
connection_string = f'{driver}://{username}:{password}@{host}/{db}'
connection=create_engine(connection_string)
engine = create_engine(connection_string)

In [19]:
suicides.to_sql("suicides1", con=connection, if_exists='replace')

In [20]:
suicides_year = suicides.groupby("Year").sum().sort_values("Num_Suicides", ascending=False)

In [21]:
suicides_year = suicides_year.reset_index()

In [22]:
suicides_year.sort_values('Year', ascending = False)

,Year,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita
31,2016,15603,132101896,2147.39000,29252100111210.00000,4106420
23,2015,203640,1774657932,8253.99000,580433884300596.00000,19516008
20,2014,222984,1912057309,10306.73000,672496725417372.00000,25665252
19,2013,223199,1890161710,10663.64000,685050211812900.00000,26911368
18,2012,230160,1912812088,11101.91000,676502580180480.00000,26058300
12,2011,236484,1993362332,11367.84000,678000047330112.00000,26936148
11,2010,238702,1997297329,11843.99000,625862405486292.00000,25193196
8,2009,243487,1976228366,12176.04000,584833269596760.00000,24145248
13,2008,235447,1860620851,12145.84000,619322403842760.00000,26936208
15,2007,233408,1859564353,12410.15000,576509655876756.00000,24709620


In [23]:
suicides.dtypes

Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year            float64
GDP/Capita            int64
Generation           object
dtype: object

# NO ME CUADRA:

In [24]:
## es lo que aparece en la tabla de arriba, pero no es muy poco? 

pop15 = suicides[suicides['Year'] == 2015]['Population'].sum()

In [25]:
pop15

1774657932

# ------

In [26]:
# Q4) Males or females commit suicide more in 2015?

# QUESTION 4:

# Males or females commit suicide more in 2015?

In [27]:
# To answer this question we are going to create a dataframe for the year 2015:

suicides_2015 = suicides[suicides['Year'] == 2015]

In [28]:
# Now we are going to filter by the columns we need:

s_gender2015 = suicides_2015[['Gender','Num_Suicides']].groupby('Gender').sum()

## As we can see, the number of suicides in 2015 is higher in males than in females:


In [29]:
s_gender2015

,Num_Suicides
Gender,
female,47248
male,156392


In [30]:
# Top 10 countries with more suicides in year 2015 taking into account the number of suicides.

In [31]:
top10countriesbynum = suicides_2015[['Country','Num_Suicides']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [32]:
top10countriesbynum

,Num_Suicides
Country,
United States,44189
Russian Federation,25432
Japan,23092
Republic of Korea,13510
Brazil,11163
Germany,10088
Ukraine,7574
Mexico,6234
Poland,5420


# Now we realize that took into account the num of suicides. But if we take into account the size of the population, results can be different. Let's check:




In [33]:
top10bysize = suicides_2015[['Country','Suicides/100kPop']].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [34]:
top10bysize

,Suicides/100kPop
Country,
Republic of Korea,389.14000
Lithuania,385.48000
Uruguay,270.02000
Slovenia,264.81000
Hungary,257.27000
Latvia,251.47000
Ukraine,244.72000
Kazakhstan,238.02000
Russian Federation,236.92000


### Taking the size of the population into account, we realize that only  Russia, Ukraine and Republic of Korea appears in this new ranking.

In [35]:
# Creating an intersection by index to see which countries appears in both rankings.
idx1 = pd.Index(top10bysize.index)
idx2 = pd.Index(top10countriesbynum.index)

In [36]:
set_intersection = idx1.intersection(idx2)

In [37]:
set_intersection

Index(['Republic of Korea', 'Ukraine', 'Russian Federation'], dtype='object', name='Country')

## Now we will do the same but by gender:

### MALES : TOP 10 COUNTRIES BY NUM

In [38]:
top10malesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [39]:
top10malesbynum

,Num_Suicides
Country,
United States,33990
Russian Federation,20861
Japan,16146
Republic of Korea,9556
Brazil,8766
Germany,7403
Ukraine,6148
Mexico,4986
Poland,4697


### FEMALES : TOP 10 COUNTRIES BY NUM

In [40]:
top10femalesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [41]:
top10femalesbynum

,Num_Suicides
Country,
United States,10199
Japan,6946
Russian Federation,4571
Republic of Korea,3954
Germany,2685
Brazil,2397
Ukraine,1426
Mexico,1248
United Kingdom,1178


In [42]:
# Creating an intersection by index to see if the countries with a higher number of suicides
# are the same countries for both genres.

idgnf = pd.Index(top10femalesbynum.index)
idgnm = pd.Index(top10malesbynum.index)

In [43]:
set_intersection = idgnf.intersection(idgnm)

In [44]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### As we can see 9 out of 10 countries match

In [45]:
# Spain appears in top10 of countries where females commit suicides.

idgnf.difference(idgnm)

Index(['Spain'], dtype='object', name='Country')

In [46]:
# Poland appears in top10 of countries where males commit suicides.

idgnm.difference(idgnf)

Index(['Poland'], dtype='object', name='Country')

### MALE: TOP 10 COUNTRIES BY SIZE


In [47]:
top10malesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]


In [48]:
top10malesbysize

,Suicides/100kPop
Country,
Lithuania,323.31000
Republic of Korea,285.19000
Uruguay,223.03000
Slovenia,217.67000
Latvia,211.07000
Ukraine,206.64000
Hungary,203.39000
Russian Federation,199.26000
Kazakhstan,195.35000


### FEMALE: TOP 10 COUNTRIES BY SIZE

In [49]:
top10femalesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]


In [50]:
top10femalesbysize

,Suicides/100kPop
Country,
Republic of Korea,103.95000
Lithuania,62.17000
Japan,58.29000
Hungary,53.88000
Belgium,52.21000
Croatia,51.63000
Slovenia,47.14000
Uruguay,46.99000
Iceland,45.69000


In [51]:
# Creating an intersection by index to see if the countries with a higher number of suicides 
# by size of the population are the same countries for both genres.

idgsf = pd.Index(top10femalesbysize.index)
idgsm = pd.Index(top10malesbysize.index)

In [52]:
set_intersection = idgsf.intersection(idgsm)

In [53]:
set_intersection

Index(['Republic of Korea', 'Lithuania', 'Hungary', 'Croatia', 'Slovenia',
       'Uruguay'],
      dtype='object', name='Country')

In [54]:
# females but no males:

idgsf.difference(idgsm)

Index(['Belgium', 'Iceland', 'Japan', 'Serbia'], dtype='object', name='Country')

In [55]:
# males but not females:

idgsm.difference(idgsf)

Index(['Kazakhstan', 'Latvia', 'Russian Federation', 'Ukraine'], dtype='object', name='Country')

### In this case, the countries Republic of Korea,  Lithuania, Hungary, Croatia, Slovenia and Uruguay match for both genres.

### But Belgiun, Iceland, Japan and Serbia appear in the top 10 for females while Kazakhstan, Latvia, Russian Federation and Ukraine appear in the top 10 for males.

# QUESTION 5: 

#  Which generation has committed suicide the most/least?


In [56]:
# checking the data 

suicides_2015.head(2)

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
576,Antigua and Barbuda,2015,female,55-74 years,1,6403,15.62000,1364863037.00000,14853,Boomers
577,Antigua and Barbuda,2015,female,15-24 years,0,8561,0.00000,1364863037.00000,14853,Millenials


## Let's check the number of suicides grouped by generations:

## First, we will see which generation committed more suicides from 2000 till 2015

In [73]:
# Change 'Year' to integer so we can filter by years.
suicides_new = suicides.astype({"Year": 'int64'})

In [74]:
# This will be our filter
mask = (suicides_new['Year'] >= 2000) & (suicides_new['Year'] < 2016)

In [83]:
df = suicides_new.loc[mask]

In [84]:
# new data frame from 2000 till 2015 
df.head(2)

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
132,Albania,2000,male,25-34 years,17,232000,7.33000,3632043908.00000,1299,Generation X
133,Albania,2000,male,55-74 years,10,177400,5.64000,3632043908.00000,1299,Silent


In [100]:
df1 = df[['Generation', 'Age','Num_Suicides']]

In [101]:
df1.groupby(['Generation','Age']).sum()

Num_Suicides
Generation      Age                      
Boomers         35-54 years        923044
                55-74 years        286601
G.I. Generation 75+ years           21633
Generation X    15-24 years         31742
                25-34 years        430755
                35-54 years        485848
Generation Z    5-14 years          15906
Millenials      15-24 years        413035
                25-34 years        177128
                5-14 years          13465
Silent          55-74 years        645749
                75+ years          349860

In [102]:
# we will drop the column 'Age' because can be confuse as is taking two different ranges
# fot the same generation. The reason why it does this is because in 2000 people from the same
# generation was in another range of age.

In [103]:
df1 = df1.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [104]:
df1

,Num_Suicides
Generation,
Boomers,1209645
Silent,995609
Generation X,948345
Millenials,603628
G.I. Generation,21633
Generation Z,15906


### ----- COUNT CUANTOS DE CADA GENERATION HAY ------

In [106]:
df2 = df.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [109]:
df2 = df2[['Population', 'Suicides/100kPop']]

In [110]:
df2

,Population,Suicides/100kPop
Generation,,
Boomers,7112476453,36854.62000
Silent,5076891426,84449.40000
Generation X,6908760170,36966.83000
Millenials,8025200090,30024.97000
G.I. Generation,86361616,4461.89000
Generation Z,2503541842,944.18000


In [111]:
df2['Population'].sum()

29713231597

In [66]:
# Let's see what happened in 2015:

In [67]:
sui_gen = suicides_2015[['Generation', 'Age','Num_Suicides']]

In [68]:
sui_gen.groupby(['Generation','Age']).sum()

Num_Suicides
Generation   Age                      
Boomers      55-74 years         53588
Generation X 35-54 years         71298
Generation Z 5-14 years           1681
Millenials   15-24 years         21841
             25-34 years         32118
Silent       75+ years           23114

## - - FALTA MERGE MILLENIALS ROWS - -

In [69]:
#Q6) Countries with the highest 5-14 years range suicide rate?

In [112]:
suicides.head()

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
0,Albania,1987,male,15-24 years,21,312900,6.71000,2156624900.00000,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19000,2156624900.00000,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83000,2156624900.00000,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59000,2156624900.00000,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28000,2156624900.00000,796,Boomers


In [ ]:
suicides_lrange = suicides

In [70]:
# As the number of suicides does not take into account how big a country is in terms of population;
# we will calculate the top10 countries by number of suicides by poplation.

top10countries['Suicides/Pop'] = top10countries['Num_Suicides']/top10countries['Population']*100000 
top10countriesbysize = suicides_2015[['Country','Num_Suicides','Population']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]
suicides_2015['Suicides/Pop'] = suicides_2015['Num_Suicides']/suicides_2015['Population']*100000 

# Now we can see that if we take into account the number of population by country
top10countries.sort_values('Suicides/Pop', ascending = False)
topcountries = top10countries[['Country','Num_Suicides','Suicides_Pop']]

NameError: name 'top10countries' is not defined